In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

C:\Users\HCILab\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(777)
mnist=input_data.read_data_sets('MNIST_data/', one_hot=True)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self):
        with tf.variable_scope(self.name):
            self.training = tf.placeholder(tf.bool)
            
            self.X = tf.placeholder(tf.float32, [None, 784])
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            
            ## Conv 1 layer 
            conv1 = tf.layers.conv2d(X_img, 32, [3, 3],
                                     padding='SAME', activation=tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(conv1, [2, 2], [2, 2],
                                           padding='SAME')
            drop1 = tf.layers.dropout(pool1, 0.7, training =self.training)
            
            ## Conv 2 layer 
            conv2 = tf.layers.conv2d(drop1, 64, [3, 3],
                                     padding='SAME', activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(conv2, [2, 2], [2, 2],
                                           padding='SAME')
            drop2 = tf.layers.dropout(pool2, 0.7, training =self.training)
            
            ## Conv 3 layer 
            conv3 = tf.layers.conv2d(drop2, 128, [3, 3],
                                     padding='SAME', activation=tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(conv3, [2, 2], [2, 2],
                                           padding='SAME')
            drop3 = tf.layers.dropout(pool3, 0.7, training =self.training)
            
            flat = tf.layers.flatten(drop3)
            full4 = tf.layers.dense(flat, 625, activation=tf.nn.relu)
            drop4 = tf.layers.dropout(full4, 0.5, training=self.training)
            
            self.logits = tf.layers.dense(drop4, 10)
            
            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            logits= self.logits, labels= self.Y))
            self.optimizer = tf.train.AdamOptimizer(
                learning_rate=learning_rate).minimize(self.cost)
            
            corrected_prediction = tf.equal(
            tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(corrected_prediction, tf.float32))
            
    def predict(self, x_test, training=False):
        return self.sess.run(self.logits,
                            feed_dict={self.X: x_test, self.training: training})
    
    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy,
                            feed_dict={self.X:x_test,
                                      self.Y: y_test, self.training: training})
    
    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer],
                             feed_dict={self.X:x_data, self.Y:y_data,
                                       self.training:training})

sess=tf.Session()

models =[]
num_models = 2

for m in range(num_models):
    models.append(Model(sess, 'model' + str(m)))
    

sess.run(tf.global_variables_initializer())

print('Learning Started !!')

for epoch in range(training_epochs):
    avg_cost = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples/batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost[m_idx] += c / total_batch
            
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', avg_cost)
print('Learning Finished!')


Learning Started !!
Epoch: 0001 cost = [0.81848643 0.76761442]
Epoch: 0002 cost = [0.3056006  0.28274763]
Epoch: 0003 cost = [0.23312731 0.22635201]
Epoch: 0004 cost = [0.20356958 0.19750656]
Epoch: 0005 cost = [0.18064287 0.18549961]
Epoch: 0006 cost = [0.17190514 0.17390538]
Epoch: 0007 cost = [0.16352931 0.16309679]
Epoch: 0008 cost = [0.15876705 0.15620522]
Epoch: 0009 cost = [0.15436599 0.15073826]
Epoch: 0010 cost = [0.14715566 0.14401712]
Epoch: 0011 cost = [0.14736836 0.14602102]
Epoch: 0012 cost = [0.14208572 0.14265901]
Epoch: 0013 cost = [0.13512134 0.13909798]
Epoch: 0014 cost = [0.13151138 0.1399959 ]
Epoch: 0015 cost = [0.13839574 0.1396395 ]
Learning Finished!


In [4]:
test_size = len(mnist.test.labels)
predictions = np.zeros([test_size, 10])
for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy:', m.get_accuracy(mnist.test.images, mnist.test.labels))
    p = m.predict(mnist.test.images)
    predictions += p
    
ensemble_correct_prediction = tf.equal(
    tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
ensemble_accuracy = tf.reduce_mean(
    tf.cast(ensemble_correct_prediction, tf.float32))

print('Ensemble accuracy:', sess.run(ensemble_accuracy))

0 Accuracy: 0.9898
1 Accuracy: 0.9879
Ensemble accuracy: 0.9905
